# QFT / PCA Spectral Decoder — Prototype
Load run logs (JSONL), compute simple DFT features over turn embeddings, and reduce with PCA to produce spectral motif cards.

In [ ]:
import json, numpy as np, os, glob\nfrom pathlib import Path\nfrom sklearn.decomposition import PCA\nfrom numpy.fft import rfft\n\n# Stub: load toy turn vectors from a JSONL file\ndef load_runs(path):\n    X = []\n    for p in glob.glob(os.path.join(path, '*.jsonl')):\n        with open(p) as f:\n            turns = [json.loads(line)['vec'] for line in f]\n            X.append(np.array(turns))\n    return X\n\ndef spectral_features(run_mat):\n    # run_mat: T x d\n    # windowless DFT per dim (prototype)\n    spec = np.abs(rfft(run_mat, axis=0))\n    # aggregate power across dims\n    power = spec.mean(axis=1)  # freq_bins\n    return power\n\ndef cards_from_runs(runs):\n    feats = [spectral_features(r) for r in runs]\n    # pad to equal length\n    L = max(len(f) for f in feats)\n    feats = [np.pad(f, (0, L - len(f))) for f in feats]\n    M = np.vstack(feats)\n    pca = PCA(n_components=8).fit(M)\n    Z = pca.transform(M)\n    cards = [{"card_id": f"card_{i}", "z": Z[i].tolist()} for i in range(len(feats))]\n    return cards\n\n# Example usage (expects qft/logs/*.jsonl with {'vec': [...]} per line)\nlogs_dir = str(Path(__file__).parent / 'logs')\nos.makedirs(logs_dir, exist_ok=True)\nruns = load_runs(logs_dir) if os.listdir(logs_dir) else [np.random.randn(128, 32) for _ in range(4)]\ncards = cards_from_runs(runs)\ncards[:2]